# Quickstart

In [1]:
import datetime
import json

import pandas

from blp import blp

In [2]:
bquery = blp.BlpQuery().start()

## Excel like functionality

In [3]:
bquery.bdh(
    ["SPY US Equity", "TLT US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)

,date,security,PX_LAST,VOLUME
0,2019-01-02,SPY US Equity,250.18,126925199.0
1,2019-01-02,TLT US Equity,122.15,19841527.0
2,2019-01-03,SPY US Equity,244.21,144140692.0
3,2019-01-03,TLT US Equity,123.54,21187045.0
4,2019-01-04,SPY US Equity,252.39,142628834.0
5,2019-01-04,TLT US Equity,122.11,12970226.0
6,2019-01-07,SPY US Equity,254.38,103139100.0
7,2019-01-07,TLT US Equity,121.75,8498104.0
8,2019-01-08,SPY US Equity,256.77,102512587.0
9,2019-01-08,TLT US Equity,121.43,7737103.0


In [4]:
bquery.bdp(["CL1 Comdty"], ["NAME", "PX_LAST"])

,security,NAME,PX_LAST
0,CL1 Comdty,Generic 1st 'CL' Future,62.7


In [5]:
bquery.bds("BCOM Index", "INDX_MWEIGHT")

,Member Ticker and Exchange Code,Percentage Weight
0,BOZ1,3.83
1,C Z1,5.46
2,CLX1,8.86
3,COX1,7.30
4,CTZ1,1.54
5,GCZ1,11.96
6,HGZ1,5.17
7,HOX1,2.34
8,KCZ1,3.60
9,KWZ1,1.68


### Using overrides

Various fields can be overriden. For info on what fields support what overrides, the best place to check is using
`FLDS` from the terminal

In [6]:
pandas.concat(
    [
        bquery.bdp(["IBM US Equity"], ["CRNCY_ADJ_PX_LAST"]),
        bquery.bdp(
            ["IBM US Equity"], ["CRNCY_ADJ_PX_LAST"], overrides=[("EQY_FUND_CRNCY", "EUR")]
        )
    ],
    axis=1,
    keys=["default", "override"]
)

default                         override                  
        security CRNCY_ADJ_PX_LAST       security CRNCY_ADJ_PX_LAST
0  IBM US Equity            139.24  IBM US Equity           118.978

### Using sedols

In [7]:
bquery.bdp(
    ["SPY US Equity", "TLT US Equity"],
    ["ID_SEDOL1"],
)

,security,ID_SEDOL1
0,SPY US Equity,2840215
1,TLT US Equity,2971546


In [8]:
bquery.bdh(
    ["/sedol1/2840215", "/sedol1/2971546"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)

,date,security,PX_LAST,VOLUME
0,2019-01-02,/sedol1/2840215,250.18,126925199.0
1,2019-01-02,/sedol1/2971546,122.15,19841527.0
2,2019-01-03,/sedol1/2840215,244.21,144140692.0
3,2019-01-03,/sedol1/2971546,123.54,21187045.0
4,2019-01-04,/sedol1/2840215,252.39,142628834.0
5,2019-01-04,/sedol1/2971546,122.11,12970226.0
6,2019-01-07,/sedol1/2840215,254.38,103139100.0
7,2019-01-07,/sedol1/2971546,121.75,8498104.0
8,2019-01-08,/sedol1/2840215,256.77,102512587.0
9,2019-01-08,/sedol1/2971546,121.43,7737103.0


## Troubleshooting

The most common type of error are errors based on bad input to the underlying `blpapi` service. Since `blp` is simply a wrapper on top of `blpapi`, these errors will be progagated to the user. The easiest way for resolving these is to inspect the relevant error message for any obvious problems followed by contacting the Bloomberg help desk.

For example, the following code will return an error since it contains an invalid ticker

```python
bquery.bdh(
    ["NOT_A_TICKER US Equity", "SPY US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)
```
```
TypeError: Response for 'NOT_A_TICKER US Equity' contains securityError
{
    "securityError": {
        "source": "3923::bbdbh4",
        "code": 15,
        "category": "BAD_SEC",
        "message": "Unknown/Invalid securityInvalid Security [nid:3923] ",
        "subcategory": "INVALID_SECURITY",
    }
}
```

If you want to ignore errors, you can instantiate a parser as follows

In [9]:
bquery = blp.BlpQuery(parser=blp.BlpParser(raise_security_errors=False)).start()

bquery.bdh(
    ["NOT_A_TICKER US Equity", "SPY US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)

Response for 'NOT_A_TICKER US Equity' contains securityError {'securityError': {'source': '139::bbdbh4', 'code': 15, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid securityInvalid Security [nid:139] ', 'subcategory': 'INVALID_SECURITY'}}


,date,security,PX_LAST,VOLUME
0,2019-01-02,SPY US Equity,250.18,126925199.0
1,2019-01-03,SPY US Equity,244.21,144140692.0
2,2019-01-04,SPY US Equity,252.39,142628834.0
3,2019-01-07,SPY US Equity,254.38,103139100.0
4,2019-01-08,SPY US Equity,256.77,102512587.0
5,2019-01-09,SPY US Equity,257.97,95006554.0
6,2019-01-10,SPY US Equity,258.88,96823923.0


## Retrieve json data

### Query generation utility methods

In [10]:
query = blp.create_query(
    request_type="HistoricalDataRequest",
    values={
        "securities": ["SPY US Equity"],
        "fields": ["VOLUME"],
        "startDate": "20190101",
        "endDate": "20190105",
    },
)
print(json.dumps(query, indent=2))

{
  "HistoricalDataRequest": {
    "securities": [
      "SPY US Equity"
    ],
    "fields": [
      "VOLUME"
    ],
    "startDate": "20190101",
    "endDate": "20190105"
  }
}


In [11]:
# deal with Timestamps for pretty printing for response

def ts_to_json(obj):
    return obj.strftime("%Y-%m-%dT%H:%M:%S.%f%z")


def time_to_json(obj):
    return obj.strftime("%H:%M:%S")


class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, pandas.Timestamp):
            return ts_to_json(obj)
        if isinstance(obj, datetime.time):
            return time_to_json(obj)
        return super().default(obj)

In [12]:
resp = bquery.query(query, parse=False, collector=list)
print(json.dumps(resp, indent=2, cls=CustomJSONEncoder))

[
  {
    "eventType": 5,
    "eventTypeName": "blpapi.Event.RESPONSE",
    "messageNumber": 0,
    "message": {
      "fragmentType": 0,
      "correlationIds": [
        25
      ],
      "messageType": "HistoricalDataResponse",
      "timeReceived": null,
      "element": {
        "HistoricalDataResponse": {
          "securityData": {
            "security": "SPY US Equity",
            "eidData": [],
            "sequenceNumber": 0,
            "fieldExceptions": [],
            "fieldData": [
              {
                "fieldData": {
                  "date": "2019-01-02T00:00:00.000000",
                  "VOLUME": 126925199.0
                }
              },
              {
                "fieldData": {
                  "date": "2019-01-03T00:00:00.000000",
                  "VOLUME": 144140692.0
                }
              },
              {
                "fieldData": {
                  "date": "2019-01-04T00:00:00.000000",
                  "VOLUME": 14262883

## Support for context manager

In [13]:
with blp.BlpQuery() as bq:
    df = bq.bdh(
        ["GME US Equity"],
        ["PX_LAST", "VOLUME"],
        start_date="20210101",
        end_date="20210130",
    )
df

,date,security,PX_LAST,VOLUME
0,2021-01-04,GME US Equity,17.25,10022474.0
1,2021-01-05,GME US Equity,17.37,4961457.0
2,2021-01-06,GME US Equity,18.36,6056248.0
3,2021-01-07,GME US Equity,18.08,6129276.0
4,2021-01-08,GME US Equity,17.69,6481960.0
5,2021-01-11,GME US Equity,19.94,14927612.0
6,2021-01-12,GME US Equity,19.95,7060665.0
7,2021-01-13,GME US Equity,31.40,144501736.0
8,2021-01-14,GME US Equity,39.91,93717410.0
9,2021-01-15,GME US Equity,35.50,46866358.0


## Stream data

In [14]:
with blp.BlpStream() as bs:
    bs.subscribe({"USDCAD Curncy": {"fields": ["LAST_PRICE"]}})
    n = 0
    for ev in bs.events(timeout=60):
        print(json.dumps(ev, indent=2, cls=CustomJSONEncoder))
        n += 1
        if n > 1:
            break





SESSION_STATUS - Message 0 - SessionConnectionDown = {
    server = "localhost:8194"
}



{
  "fragmentType": 0,
  "correlationIds": [
    "USDCAD Curncy"
  ],
  "messageType": "MarketDataEvents",
  "timeReceived": "2021-08-20T16:50:17.396011+0000",
  "element": {
    "MarketDataEvents": {
      "MKTDATA_EVENT_TYPE": "SUMMARY",
      "MKTDATA_EVENT_SUBTYPE": "INITPAINT",
      "BID": 1.2839,
      "ASK": 1.284,
      "BEST_BID": 1.2839,
      "BEST_ASK": 1.284,
      "BID_ALL_SESSION": 1.2839,
      "ASK_ALL_SESSION": 1.284,
      "IND_BID_FLAG": false,
      "IND_ASK_FLAG": false,
      "BID_TDY": 1.2839,
      "PRICE_LAST_BID_RT": 1.2839,
      "ASK_TDY": 1.284,
      "PRICE_LAST_ASK_RT": 1.284,
      "BID_DIR": -1,
      "ASK_DIR": 1,
      "BID2": 1.2839,
      "ASK2": 1.284,
      "BID_ALL_SESSION_TDY_RT": 1.2839,
      "ASK_ALL_SESSION_TDY_RT": 1.284,
      "CURRENT_SESSION_RT": 2,
      "PX_OPEN_ALL_WITH_SWITCHOVER_RT": 1.2827529907226562,
      "PX_CLOSE_ALL_WITH_SWITCHOVER_RT": 1.2828,
      "LAST_PRICE_ALL_SESSION_DIR_RT": 1,
      "SIMP_LAST_PX_ALL_SESS_DIR_RT": 


20AUG2021_12:50:34.721 28102:140461514245888 WARN blpapi_platformcontroller.cpp:377 blpapi.session.platformcontroller.{1} Connectivity lost, no connected endpoints. 

20AUG2021_12:50:34.722 28102:140461479098112 WARN blpapi_platformcontroller.cpp:377 blpapi.session.platformcontroller.{2} Connectivity lost, no connected endpoints. 

20AUG2021_12:50:34.724 28102:140461522839296 WARN apicm_apiconnector.cpp:570 ApiConnector::negotiatorCallback localhost: Session negotiation failed for 127.0.0.1:8194 result = 1 

20AUG2021_12:50:34.724 28102:140461487490816 WARN apicm_apiconnector.cpp:570 ApiConnector::negotiatorCallback localhost: Session negotiation failed for 127.0.0.1:8194 result = 1 

20AUG2021_12:50:34.724 28102:140461522839296 WARN blpapi_apicmadapter.cpp:335 blpapi.session.transporttcp.apicmadapter.{1} Failed BBCOMM session negotiation localhost:8194 <127.0.0.1:8194> 

20AUG2021_12:50:34.724 28102:140461487490816 WARN blpapi_apicmadapter.cpp:335 blpapi.session.transporttcp.apicmada